In [16]:
import pandas as pd
import numpy as np
import pickle
import logging
from multiprocessing import Pool, cpu_count
from tqdm import tqdm as tqdm_notebook 
from tqdm import trange
from tools import *
import convert_examples_to_features
import torch
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,\
                              TensorDataset
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

In [2]:
DATA_DIR = "./data/"
OUTPUT_DIR = './outputs/'
LOG_DIR = "./logs/"
SAVES_DIR = "./saves/"
CACHE_DIR = "./cache/"
WEIGHTS_NAME = "pytorch_model.bin"
CONFIG_NAME = "config.json"

MAX_SEQ_LENGTH = 128
TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'


In [3]:
def cleaner(x):
    return x.replace(r'\n', ' ', regex=True)

def to_tsv(X, filename = "train"):
    prepro_X = pd.DataFrame({
        'id': range(X.shape[0]),
        'label': (X.choose_one == "Relevant")*1,
        "alpha": ['a']*X.shape[0],
        'text': cleaner(X.text)
    })
    prepro_X.to_csv(DATA_DIR + filename +'.tsv', sep='\t', index=False, header=False)
    return prepro_X



In [4]:
X = pd.read_excel("./Classeur1.xlsx")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, X_test, y_train, y_test = train_test_split(
X, X.choose_one, test_size=0.2, random_state=42)
to_tsv(X_train, "train")
to_tsv(X_test, "dev")

,id,label,alpha,text
10219,0,1,a,The sunset looked like an erupting volcano ......
1413,1,1,a,#7294 Nikon D50 6.1 MP Digital SLR Camera Body...
8887,2,0,a,Mental/Twitter Note: Make sure my smoke alarm ...
4505,3,0,a,?????? EMERGENCY ?????? NEED PART 2 and 3!!! #...
8765,4,0,a,WHELEN MODEL 295SS-100 SIREN AMPLIFIER POLICE ...
...,...,...,...,...
2807,2171,1,a,'The Big Ten has their annual football media d...
8685,2172,0,a,Does that sewer look like it's sinking to you?...
8900,2173,0,a,sorry-I built a fire by my desk already. RT@ir...
1543,2174,0,a,@CranBoonitz So going to make any bomb threats...


,id,label,alpha,text
10219,0,1,a,The sunset looked like an erupting volcano ......
1413,1,1,a,#7294 Nikon D50 6.1 MP Digital SLR Camera Body...
8887,2,0,a,Mental/Twitter Note: Make sure my smoke alarm ...
4505,3,0,a,?????? EMERGENCY ?????? NEED PART 2 and 3!!! #...
8765,4,0,a,WHELEN MODEL 295SS-100 SIREN AMPLIFIER POLICE ...
...,...,...,...,...
2807,2171,1,a,'The Big Ten has their annual football media d...
8685,2172,0,a,Does that sewer look like it's sinking to you?...
8900,2173,0,a,sorry-I built a fire by my desk already. RT@ir...
1543,2174,0,a,@CranBoonitz So going to make any bomb threats...


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [6]:
train_examples[0].text_a

'i dont even remember slsp happening i just remember being like wtf and then the lights turned off and everyone screamed for the encore'

In [7]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [17]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 8700 examples..
Spawning 15 processes..
100%|██████████| 8700/8700 [01:57<00:00, 74.24it/s]


In [18]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [19]:
with open(DATA_DIR + "train_features.pkl", "rb") as f:
    train_features = pickle.load(f)

In [20]:
BERT_MODEL = 'bert-base-cased'
# model = BertForSequenceClassification.from_pretrained(CACHE_DIR + 'cased_base_bert_pytorch.tar.gz', cache_dir=CACHE_DIR, num_labels=num_labels)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)

In [21]:
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)


In [22]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [23]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)


all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

In [24]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [25]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        
        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]
0.761897
0.767114
0.673255
0.857645
0.531135
0.789837
0.732031
0.641195
0.657216
0.633697
0.600990
0.643154
0.503343
0.684762
0.487321
0.573634
0.438752
0.588914
0.429721
0.555490
0.519320
0.467452
0.483013
0.437010
0.503619
0.350348
0.423336
0.477311
0.410641
0.538209
0.349074
0.910564
0.652847
0.818182
0.457333
0.561753
0.476739
0.530877
0.884277
0.400132
0.522512
0.420677
0.516149
0.534801
0.510714
0.510517
0.420445
0.604194
0.270513
0.376185
0.500917
0.543613
0.609647
0.498230
0.588977
0.476556
0.447502
0.722523
0.547739
0.308393
0.395038
0.390344
0.365341
0.284489
0.625509
0.603654
0.431675
0.287984
0.435959
0.459571
0.579084
0.575802
0.562798
0.488484
0.467944
0.428593
0.528826
0.455367
0.239988
0.413966
0.395496
0.353190
0.651343
0.694934
0.433135
0.468799
0.565185
0.342763
0.523616
0.455034
0.315365
0.428565
0.457824
0.355268
0.532931
0.478992
0.551152
0.637531
0.445324
0.471705
0.537642
0.578321
0.400740
0.523341
0.585868
0.252216
0

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
8491,778253309,False,finalized,5,8/27/15 16:07,Not Relevant,1.0000,NaN,screamed,NaN,i dont even remember slsp happening i just rem...,6.291070e+17,2.327739e+08
7178,778251995,False,finalized,5,8/27/15 20:16,Not Relevant,1.0000,NaN,mudslide,Edinburgh,@hazelannmac ooh now I feel guilty about wishi...,6.290180e+17,2.750220e+07
2423,778247239,False,finalized,5,8/30/15 0:15,Not Relevant,1.0000,NaN,collide,planeta H2o,Soultech - Collide (Club Mix) http://t.co/8xIx...,6.290920e+17,6.052387e+08
10612,778255430,False,finalized,5,8/27/15 17:03,Relevant,0.7978,NaN,wounded,NaN,Police Officer Wounded Suspect Dead After Exch...,6.291190e+17,2.305930e+09
10791,778255609,False,finalized,5,8/27/15 22:11,Not Relevant,1.0000,NaN,wrecked,Sunny Southern California,Cramer: Iger's 3 words that wrecked Disney's s...,6.290800e+17,2.464266e+07


In [37]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

'./outputs/vocab.txt'

In [26]:
def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [27]:
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

In [28]:
processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

In [29]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

In [23]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 2176 examples..
Spawning 15 processes..


In [24]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

In [25]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)

In [26]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [13]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(OUTPUT_DIR, cache_dir=CACHE_DIR, num_labels=len(label_list))

In [19]:
model.to(device)
0

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [27]:
from sklearn.metrics import matthews_corrcoef, confusion_matrix

In [28]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    loss_fct = CrossEntropyLoss()
    tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
if OUTPUT_MODE == "classification":
    preds = np.argmax(preds, axis=1)
elif OUTPUT_MODE == "regression":
    preds = np.squeeze(preds)
result = compute_metrics("test", all_label_ids.numpy(), preds)

result['eval_loss'] = eval_loss

output_eval_file = os.path.join("./logs/", "eval_results.txt")

with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in (result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

In [31]:
"Do you like pasta?" in X.text

False

In [29]:
result = compute_metrics("test", all_label_ids.numpy(), preds)

result['eval_loss'] = eval_loss

output_eval_file = os.path.join("./logs/", "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in (result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))